# softmax回归的简洁实现
使用pytorch中的高级API
线性回归变得更加容易。 同样，通过深度学习框架的高级API也能更方便地实现softmax回归模型。
1. 初始化batch_size=256, 读取train & test数据集
2. softmax从输入到输出是一个全连接层, 在本例中输入和输出的维数是784x10，因此们需要在Sequential中添加一个带有10个输出的全连接层，我们仍然以均值0和标准差0.01随机初始化权重。另一方面，原始输入是28x28的灰度值矩阵，因此需要展平为向量，即flatten层
3. **交叉熵损失**。预测值在对应分类的负对数值，预测值通过特征的线性组合外套一个softmax操作得到，即$\hat y_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$。这里其实有个scaling的问题, 假设$o_k$中的一些数值非常大，$\exp(o_k)$也会很大, 可能上溢, 出现inf,nan等情况.一种思路是$$\begin{split}\begin{aligned}
\hat y_j & =  \frac{\exp(o_j - \max(o_k))\exp(\max(o_k))}{\sum_k \exp(o_k - \max(o_k))\exp(\max(o_k))} \\
& = \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}.
\end{aligned}\end{split}$$另一方面损失函数(注意取其相反数)为$$\begin{split}\begin{aligned}
\log{(\hat y_j)} & = \log\left( \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}\right) \\
& = \log{(\exp(o_j - \max(o_k)))}-\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)} \\
& = o_j - \max(o_k) -\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)}.
\end{aligned}\end{split}$$


---
4. 优化函数选择SGD.实际上交叉熵损失也是一行代码
5. 进行训练

In [3]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST
# 1. load data 
batch_size = 256
trans = transforms.ToTensor()
train_minist = FashionMNIST(root="../../data", train=True, transform=trans)
test_minist = FashionMNIST(root="../../data", train=False, transform=trans)
train_iter = DataLoader(train_minist, batch_size, shuffle=True, num_workers=6)
test_iter = DataLoader(test_minist, batch_size, shuffle=False, num_workers=6)
next(iter(train_iter))[0].shape

torch.Size([256, 1, 28, 28])

In [4]:
# 2. 定义模型
from torch import nn
num_inputs = 28*28
num_outputs = 10
net = nn.Sequential(nn.Flatten(), nn.Linear(num_inputs, num_outputs))
# 3. 参数初始化 -- 这里将采用常用的初始化方法 apply
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.03)
net.apply(init_weights)
# 3. 损失函数
loss = nn.CrossEntropyLoss(reduction='none')
# 4. 优化函数
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# 5. 进行训练
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        trainer.zero_grad()  # 注意将原先的计算图清空
        l.mean().backward()  # 反向传播
        trainer.step()  # 更新梯度
    
# 6. 计算精度
def cmpY(y_hat, y):
    return (y == y_hat.argmax(dim=1)).sum()
acc_cnt = 0
sam_tot = 0
for X, y in test_iter:
    acc_cnt += cmpY(softmax(X), y)
    sam_tot += len(y)
acc_cnt, sam_tot, acc_cnt/sam_tot

NameError: name 'softmax' is not defined

1. [nn.init 中实现的初始化函数 uniform, normal, const](https://cloud.tencent.com/developer/article/1627511)
<br>
2. [每天学点pytorch--torch.nn.Module的apply()方法](https://blog.csdn.net/qiumokucao/article/details/121356553)apply(fn)的官网介绍，该方法会将fn递归的应用于模块的每一个子模块（.children()的结果）及其自身。典型的用法是，对一个model的参数进行初始化
<br>
3. [nn.CrossEntropyLoss]()